In [14]:
import polars as pl
from db.model import Tactics, Techniques, TacTech, Datasources, TechData, DataCollection, Cyberkillchain, AdditionalDatasources, Mitigations, TechMitigations, Detections
from miter.create_db import create_tactic_as_records, create_technique_as_records, create_mitigation_as_records

In [15]:
tactic_path = "/home/work/data/enterprise-attack-v15.1-tactics.xlsx"
datasource_path = "/home/work/data/enterprise-attack-v15.1-datasources.xlsx"
technique_path = "/home/work/data/enterprise-attack-v15.1-techniques.xlsx"
mitigation_path = "/home/work/data/enterprise-attack-v15.1-mitigations.xlsx"
relation_path = "/home/work/data/enterprise-attack-v15.1-relationships.xlsx"

In [16]:
target_model = Detections

In [17]:
target_model.drop_table()
target_model.create_table()

In [18]:
COLUMN_MAPPING_MITIGATION = {
    "target ID": "technique_id",
    "ID": "datasource_id",
    "mapping description": "detection_en",
}

In [19]:
relation_df = pl.read_excel(relation_path)
relation_df = relation_df.filter(pl.col("mapping type")=="detects")
datasource_df = pl.read_excel(datasource_path)
datasource_df = datasource_df.with_columns(pl.col("name_sub").str.strip_chars())
# datasource_path

In [20]:
df = relation_df.join(datasource_df, how="left", left_on= "source name", right_on="name_sub")
df = df.select(list(COLUMN_MAPPING_MITIGATION.keys())).rename(COLUMN_MAPPING_MITIGATION)
df = df.with_columns(pl.Series("detection_jp", [None]* len(df))).sort(by=["technique_id", "datasource_id"])

/tmp/ipykernel_2702413/1213803955.py:1: DeprecationWarning: The default coalesce behavior of left join will change to `False` in the next breaking release. Pass `coalesce=True` to keep the current behavior and silence this warning.
  df = relation_df.join(datasource_df, how="left", left_on= "source name", right_on="name_sub")


In [21]:
ones = df.to_dicts()

In [22]:
ones

[{'technique_id': 'T1001',
  'datasource_id': 'DS0029.002',
  'detection_en': 'Monitor and analyze traffic patterns and packet inspection associated to protocol(s), leveraging SSL/TLS inspection for encrypted traffic, that do not follow the expected protocol standards and traffic flows (e.g extraneous packets that do not belong to established flows, gratuitous or anomalous traffic patterns, anomalous syntax, or structure). Consider correlation with process monitoring and command line to detect anomalous processes execution and command line arguments associated to traffic patterns (e.g. monitor anomalies in use of files that do not normally initiate connections for respective protocol(s)).',
  'detection_jp': None},
 {'technique_id': 'T1001.001',
  'datasource_id': 'DS0029.002',
  'detection_en': 'Monitor and analyze traffic patterns and packet inspection associated to protocol(s), leveraging SSL/TLS inspection for encrypted traffic, that do not follow the expected protocol standards an

In [23]:
ones[:2]

[{'technique_id': 'T1001',
  'datasource_id': 'DS0029.002',
  'detection_en': 'Monitor and analyze traffic patterns and packet inspection associated to protocol(s), leveraging SSL/TLS inspection for encrypted traffic, that do not follow the expected protocol standards and traffic flows (e.g extraneous packets that do not belong to established flows, gratuitous or anomalous traffic patterns, anomalous syntax, or structure). Consider correlation with process monitoring and command line to detect anomalous processes execution and command line arguments associated to traffic patterns (e.g. monitor anomalies in use of files that do not normally initiate connections for respective protocol(s)).',
  'detection_jp': None},
 {'technique_id': 'T1001.001',
  'datasource_id': 'DS0029.002',
  'detection_en': 'Monitor and analyze traffic patterns and packet inspection associated to protocol(s), leveraging SSL/TLS inspection for encrypted traffic, that do not follow the expected protocol standards an

In [24]:
target_model.insert_many(ones).execute()

0

In [12]:
target_model

<Model: Detections>